## VERSION 2


In [ ]:
from openai import OpenAI
import ast

# Set up OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  # Replace with your actual key
)

# STEP 1 – Generate Task
def generate_task(goal):
    prompt = f"""
The user is learning: "{goal}".
Generate a short but meaningful persuasive or reflective writing task (3–5 sentences)
that helps assess:
- vocabulary use
- curiosity
- confidence
- compression
- emotional tone

Return just the task instructions, not explanations.
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are a helpful AI tutor."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# STEP 2 – Score User Response
def score_response(goal, task, user_response):
    prompt = f"""
The user is learning: "{goal}"
They were asked to complete this writing task: "{task}"
They wrote the following:

{user_response}

Please rate them between 0.0 and 1.0 on:
1. Vocabulary Expansion
2. Curiosity
3. Confidence
4. Output Compression
5. Emotional Consistency

Return only a Python dictionary like:
{{"vocabulary_expansion_count": ..., "curiosity_level": ..., "confidence_level": ..., "output_compression_ratio": ..., "emotional_consistency_score": ...}}
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are an AI evaluator. Return only a Python dictionary."},
            {"role": "user", "content": prompt}
        ]
    )
    return ast.literal_eval(response.choices[0].message.content)

# STEP 3 – Generate Feedback
def give_feedback(goal, scores):
    prompt = f"""
The user is learning: "{goal}".
Here are their scores:
- Vocabulary Expansion Count: {scores['vocabulary_expansion_count']}
- Curiosity Level: {scores['curiosity_level']}
- Confidence Level: {scores['confidence_level']}
- Output Compression Ratio: {scores['output_compression_ratio']}
- Emotional Consistency Score: {scores['emotional_consistency_score']}

Give:
1. A natural explanation of their learning state
2. Tailored guidance to help them improve in these areas
3. Specific next-step exercises
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are an encouraging but honest AI learning coach."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# ========== ✅ RUN HERE ==========
# Step 0 – Enter Your Learning Goal
goal = input("What are you trying to learn today? ")

# Step 1 – Generate a Personalized Task
task = generate_task(goal)
print("\n📝 TASK FOR YOU TO COMPLETE:\n", task)

# Step 2 – Write Your Answer
user_response = input("\n✍️ Paste your answer here:\n")

# Step 3 – LLM Scores Your Work
scores = score_response(goal, task, user_response)
print("\n📊 SCORES:", scores)

# Step 4 – Get LLM-Powered Feedback
feedback = give_feedback(goal, scores)
print("\n💡 FEEDBACK:\n", feedback)
